<a href="https://colab.research.google.com/github/micah-shull/RAG-LangChain/blob/main/LC_016_RAG_EVAL_JudgementDay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧠 Rubric-Based Evaluation with GPT

In this notebook, we implemented an **automated evaluation framework** using a rubric-based scoring system judged by a GPT model.

### 🎯 Purpose

The goal is to assess AI-generated answers to user questions using a structured set of criteria — just like a human reviewer would.

### 📋 Rubric Dimensions

Each answer is scored on three dimensions:

1. **Factual Accuracy**  
   - Is the answer correct based on the retrieved context?

2. **Relevance**  
   - Does the answer fully address the user's question?

3. **Clarity**  
   - Is the response clearly written and easy to understand?

Each dimension is rated on a 1–5 scale, with **5 being excellent** and **1 being unacceptable**.

### 🧪 How It Works

1. For each question, we generate an answer using our RAG pipeline.
2. We also capture the top-`k` retrieved context chunks used to generate that answer.
3. We send the `question`, `context`, and `answer` to a GPT model along with a detailed rubric.
4. The GPT model returns:
   - Numerical scores for **factuality**, **relevance**, and **clarity**
   - A short **comment per dimension** explaining the score

### 💡 Why This Works

- GPT is capable of applying structured, rubric-based judgments consistently when given clear instructions.
- This method allows us to evaluate our RAG system **at scale**, with nuanced and explainable feedback.
- It bridges the gap between raw performance and real-world quality metrics (helpfulness, correctness, clarity).

We validated this evaluator using flawed responses and confirmed that it behaves **critically and appropriately**, rather than just being generous with scores.


---

### ✅ Solutions Implemented

* Built a clean RAG chain that retrieves top-`k` chunks and generates answers using GPT-4.
* Captured both the **generated answer** and the **retrieved context** for each question.
* Designed an evaluation function that passed the **same context** to an LLM-based evaluator.
* Updated the evaluator prompt to trust document context and ignore its own outdated knowledge.

---

## ✅ Evaluation Sanity Check: Validating the GPT Judge

Before trusting our rubric-based evaluation results, we tested whether the evaluator could **reliably detect poor answers** — not just rubber-stamp everything with perfect scores.

### 🔍 Method

We selected three of our original questions and manually substituted **intentionally flawed answers** that were:

- Factually incorrect
- Irrelevant to the question
- Or overly vague

We kept the **retrieved context unchanged**, ensuring that only the answer varied. This isolated the evaluation process.

### 🧪 Flawed Examples

| Case | Type of Flaw | Summary of Flawed Answer |
|------|---------------|--------------------------|
| 1 | Irrelevant | Mentioned tech startups and basketball, not economic indicators |
| 2 | Contradictory | Claimed CPI was stable, ignoring a known increase |
| 3 | Reversed fact | Claimed consumer confidence rose when it actually dropped |

### ✅ Results

The evaluator responded appropriately:

- Gave **1–2** scores for factuality and relevance where appropriate
- Gave **3–4** for clarity when writing was decent but content was wrong
- Provided **dimension-specific feedback** in structured comments

### 📌 Conclusion

This validation confirms that:

- The rubric prompt is working as intended
- The GPT-based judge can reliably **score answers across multiple dimensions**
- Earlier perfect 5/5/5 scores on good answers were **earned, not inflated**

This gives us confidence to use this evaluator for:
- Pipeline quality tracking
- Prompt iteration
- Comparative model testing




## Pip Install Packages

In [1]:
!pip install --upgrade --quiet \
    langchain \
    langchain-huggingface \
    langchain-openai \
    langchain-community \
    chromadb \
    python-dotenv \
    transformers \
    accelerate \
    sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 6.4 MB/s eta 0:00:0

## Load Libaries

In [2]:
# 🌿 Environment setup
import os                                 # File paths and OS interaction
from dotenv import load_dotenv            # Load environment variables from .env file
import langchain; print(langchain.__version__)  # Check LangChain version
import itertools

# 📄 Document loading and preprocessing
from langchain_core.documents import Document                   # Base document type
from langchain_community.document_loaders import TextLoader     # Loads plain text files
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Splits long docs into smaller chunks

# 🔢 Embeddings + vector storage
from langchain_huggingface import HuggingFaceEmbeddings         # HuggingFace embedding model
from langchain.vectorstores import Chroma                       # Persistent vector DB (Chroma)

# 💬 Prompting + output
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate           # Chat-style prompt templates
from langchain_core.output_parsers import StrOutputParser       # Converts model output to string

# 🔗 Chains / pipelines
from langchain_core.runnables import Runnable, RunnableLambda   # Compose custom pipelines

# 🧠 (Optional) Hugging Face LLM client setup
# from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace  # For HF inference API

# 🧾 Pretty printing
import textwrap                         # Format long strings for printing
from pprint import pprint               # Nicely format nested data structures

# Pydantic
from pydantic import BaseModel, ValidationError

0.3.26


## SET MODEL PARAMS

In [3]:
# Load API key
from openai import OpenAI

# Load token from .env.
load_dotenv("/content/API_KEYS.env", override=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# SET MODEL PARAMS
EMBED_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 200
CHUNK_OVERLAP = 50
K = 2

LLM_MODEL = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.4  # Moderate creativity; adjust as needed
)



## 🧾 Document Cleaning

### 🧾 1. **Load the `.txt` files**

We’ll loop through all files in the folder using `TextLoader`.

### 🧹 2. **Cleaning**

Basic cleaning (e.g. stripping newlines, extra whitespace) is often helpful **before splitting**, especially if the files came from exports or copy-paste.

### ✂️ 3. **Split into chunks**

We’ll use `RecursiveCharacterTextSplitter` to chunk documents (typically 500–1000 characters with slight overlap for context continuity).

---

### 🧼 Why Basic Cleaning Helps

* Removes linebreaks and blank lines that confuse LLMs
* Avoids splitting chunks in weird places
* Standardizes format before embedding

Later you can add more advanced cleaning (e.g., remove boilerplate, normalize headers), but this is a solid default.





In [4]:
# Path to documents
docs_path = "/content/CFFC"

# Step 1: Load all .txt files in the folder
raw_documents = []
for filename in os.listdir(docs_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(docs_path, filename)
        loader = TextLoader(file_path, encoding="utf-8")
        docs = loader.load()
        raw_documents.extend(docs)

print(f"Loaded {len(raw_documents)} documents.")

# Step 2 (optional): Clean up newlines and extra whitespace
def clean_doc(doc: Document) -> Document:
    cleaned = " ".join(doc.page_content.split())  # Removes newlines & extra spaces
    return Document(page_content=cleaned, metadata=doc.metadata)

cleaned_documents = [clean_doc(doc) for doc in raw_documents]

# Step 3: Split documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

chunked_documents = splitter.split_documents(cleaned_documents)

print(f"Split into {len(chunked_documents)} total chunks.")

# Preview the first 5 chunks
print(f"Showing first 5 of {len(chunked_documents)} chunks:\n")

for i, doc in enumerate(chunked_documents[:5]):
    print(f"--- Chunk {i+1} ---")
    print(f"Source: {doc.metadata.get('source', 'N/A')}\n")
    print(textwrap.fill(doc.page_content[:500], width=100))  # limit preview to 500 characters
    print("\n")

Loaded 5 documents.
Split into 83 total chunks.
Showing first 5 of 83 chunks:

--- Chunk 1 ---
Source: /content/CFFC/CFFC_Features_UseCases.txt

# 📊 Forecasting You Can Trust in Uncertain Times We help mid-sized businesses stay ahead of sales
volatility and protect their bottom line by cutting forecast errors in half — using powerful machine


--- Chunk 2 ---
Source: /content/CFFC/CFFC_Features_UseCases.txt

forecast errors in half — using powerful machine learning models. --- ## 🚀 Key Benefits at a Glance
- Cut forecasting errors by **50% or more** - Detect changes in demand **before they impact cash


--- Chunk 3 ---
Source: /content/CFFC/CFFC_Features_UseCases.txt

changes in demand **before they impact cash flow** - Respond to opportunities with **confidence and
speed** - Replace spreadsheets with **smarter, adaptive forecasting** --- ## ⚠️ 1. Risk: Can I


--- Chunk 4 ---
Source: /content/CFFC/CFFC_Features_UseCases.txt

adaptive forecasting** --- ## ⚠️ 1. Risk: Can I Avoid Costly

## ✅ Embed + Persist in Chroma




In [5]:
# Step 1: Set up Hugging Face embedding model
embedding_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

# Step 2: Set up Chroma with persistence
persist_dir = "chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embedding_model,
    persist_directory=persist_dir
)

print(f"✅ Stored {len(chunked_documents)} chunks in Chroma at '{persist_dir}'")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Stored 83 chunks in Chroma at 'chroma_db'


## ✅ Create the Retriever & Prompt Template

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": K})

prompt_template = ChatPromptTemplate.from_template("""
You are a helpful assistant that uses business documents to answer questions.
Use the following context to answer the question as accurately as possible.

Context:
{context}

Question:
{question}

Answer:
""")

def generate_answer(question: str):
    """
    1. Retrieve K chunks
    2. Concatenate into a single context string
    3. Feed context + question to the generator LLM
    4. Return (answer, context)   <-- key!
    """
    # Step-1 ➜ retrieve top K document chunks
    docs = retriever.invoke(question)

    # Step-2 ➜ combine those into a context string
    context = "\n\n".join(doc.page_content for doc in docs)

    # Step-3 ➜ fill the prompt template with context and question
    llm_input = prompt_template.format(context=context, question=question)

    # Step-4 ➜ call the LLM and get its plain text output
    answer = StrOutputParser().invoke(LLM_MODEL.invoke(llm_input))

    # Step-5 ➜ return the answer AND the context used
    return answer, context


## ✅ Step 3: Create the RAG Chain & Run a Query!

In [19]:
# Define RAG chain
rag_chain = (
    RunnableLambda(lambda d: {
        "question": d["question"],
        "docs": retriever.invoke(d["question"])
    })
    | RunnableLambda(lambda d: {
        "context": "\n\n".join([doc.page_content for doc in d["docs"]]),
        "question": d["question"]
    })
    | prompt_template
    | LLM_MODEL
    | StrOutputParser()
)

# Invoke RAG
response = rag_chain.invoke({
    "question": "What are the recent economic indicators in Gainesville that affect local businesses?"
})

# Print response nicely
import textwrap
print("\n" + textwrap.fill(response, width=100))



The recent economic indicators in Gainesville that affect local businesses include declining
consumer confidence, softening retail sales, and rising unemployment. These factors contribute to
revenue uncertainty for businesses, even those that are typically steady. Improved forecasting can
help businesses navigate these challenges.


#EVALUATION

## Evaluation Prompt

In [23]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

evaluator_system_prompt = """
You are an evaluator. Judge whether the AI's answer is acceptable based ONLY on the retrieved context provided.

You are not limited by your own knowledge. Assume the context is reliable, even if it contains facts or dates beyond your training cutoff.

Evaluate based on:
- ✅ Factual correctness based on context
- ✅ Clarity and coherence
- ✅ Relevance to the question

Respond in strict JSON format:
{
  "is_acceptable": true or false,
  "feedback": "short explanation"
}
"""


def evaluate_response(question: str, answer: str, context: str) -> Evaluation:
    user_block = f"""
User Question:
{question}

Retrieved Context:
{context}

Agent Answer:
{answer}

Please evaluate the answer.
"""

    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user",   "content": user_block},
    ]

    raw = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.0,
    ).choices[0].message.content

    return Evaluation.model_validate_json(raw)

## Questions

In [24]:
qa_test = [
#     "What are the recent economic indicators in Gainesville that affect local businesses?",
#     "What is the recent trend in the Consumer Price Index (CPI) and how might it affect small businesses?",
    "How has the Consumer Confidence Index changed since December 2024?",
    # "What challenges might a business face if the Consumer Confidence Index drops?"
    # "How can inflation affect staffing or wage budgets for small businesses?",
    # "ow are CPI and retail sales different in their impact on business operations?",
    # "Why should a local business owner monitor federal economic indicators like CPI, retail sales, and consumer confidence?",
    # "What are three key actions a business owner can take in response to recent economic trends?"
]

## Evaluation

In [25]:
for i, q in enumerate(qa_test, 1):
    answer, context = generate_answer(q)          # <— same context for both models
    result          = evaluate_response(q, answer, context)

    print("\n" + "="*120)
    print(f"🔢 Question {i}")
    print("📌", textwrap.fill(q, width=100), "\n")
    print("📝 Answer:\n", textwrap.fill(answer, 100), "\n")
    print("✅ Acceptable:", result.is_acceptable)
    print("💬 Feedback:\n", textwrap.fill(result.feedback, 100))
    print("="*120)


🔢 Question 1
📌 How has the Consumer Confidence Index changed since December 2024? 

📝 Answer:
 The Consumer Confidence Index has dropped from **74.0 to 64.7** since December 1, 2024, reflecting a
**12.6% decline**. 

✅ Acceptable: True
💬 Feedback:
 The answer accurately reflects the information from the retrieved context regarding the change in
the Consumer Confidence Index since December 2024, providing both the numerical values and the
percentage decline.


# JUDGEMENT

## Judgement Prompt

In [26]:
rubric_prompt = """
You are an expert evaluator. Your task is to rate an AI-generated answer based on 3 criteria:

1. **Factual Accuracy** – Is the answer correct based on the retrieved context?
2. **Relevance** – Does it fully and directly answer the user's question?
3. **Clarity** – Is it well-structured and easy to understand?

You must base your judgment ONLY on the context provided — ignore anything you do not see in the context.

Respond in the following strict JSON format:
{
  "factuality": 1-5,
  "relevance": 1-5,
  "clarity": 1-5,
  "comments": "Concise explanation of your reasoning"
}
"""


## Evaluation Function

In [27]:
class RubricScore(BaseModel):
    factuality: int
    relevance: int
    clarity: int
    comments: str

def evaluate_with_rubric(question: str, answer: str, context: str) -> RubricScore:
    user_block = f"""
User Question:
{question}

Retrieved Context:
{context}

Agent Answer:
{answer}

Evaluate the answer using the rubric.
"""

    messages = [
        {"role": "system", "content": rubric_prompt},
        {"role": "user", "content": user_block}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.0,
    )

    try:
        return RubricScore.model_validate_json(response.choices[0].message.content)
    except Exception as e:
        print("❌ Failed to parse rubric response:", e)
        print("Raw response:\n", response.choices[0].message.content)
        raise


## RUN

In [28]:
for i, q in enumerate(qa_test, 1):
    answer, context = generate_answer(q)
    scores = evaluate_with_rubric(q, answer, context)

    print("\n" + "="*120)
    print(f"🔢 Question {i}")
    print("📌", textwrap.fill(q, 100), "\n")
    print("📝 Answer:\n", textwrap.fill(answer, 100), "\n")
    print(f"🎯 Scores: Factuality={scores.factuality} | Relevance={scores.relevance} | Clarity={scores.clarity}")
    print("💬 Comments:", textwrap.fill(scores.comments, 100))
    print("="*120)


🔢 Question 1
📌 How has the Consumer Confidence Index changed since December 2024? 

📝 Answer:
 The Consumer Confidence Index has dropped from 74.0 to 64.7 since December 1, 2024, reflecting a
12.6% decline. 

🎯 Scores: Factuality=5 | Relevance=5 | Clarity=5
💬 Comments: The answer accurately reflects the information from the context regarding the change in the Consumer
Confidence Index, directly answers the user's question, and is clearly stated.


In [29]:
qa_test = [
    "What are the recent economic indicators in Gainesville that affect local businesses?",
    "What is the recent trend in the Consumer Price Index (CPI) and how might it affect small businesses?",
    "How has the Consumer Confidence Index changed since December 2024?",
    # "What challenges might a business face if the Consumer Confidence Index drops?"
    # "How can inflation affect staffing or wage budgets for small businesses?",
    # "ow are CPI and retail sales different in their impact on business operations?",
    # "Why should a local business owner monitor federal economic indicators like CPI, retail sales, and consumer confidence?",
    # "What are three key actions a business owner can take in response to recent economic trends?"
]

for i, q in enumerate(qa_test, 1):
    answer, context = generate_answer(q)
    scores = evaluate_with_rubric(q, answer, context)

    print("\n" + "="*120)
    print(f"🔢 Question {i}")
    print("📌", textwrap.fill(q, 100), "\n")
    print("📝 Answer:\n", textwrap.fill(answer, 100), "\n")
    print(f"🎯 Scores: Factuality={scores.factuality} | Relevance={scores.relevance} | Clarity={scores.clarity}")
    print("💬 Comments:", textwrap.fill(scores.comments, 100))
    print("="*120)


🔢 Question 1
📌 What are the recent economic indicators in Gainesville that affect local businesses? 

📝 Answer:
 Recent economic indicators in Gainesville that affect local businesses include declining consumer
confidence, softening retail sales, and rising unemployment. These factors contribute to revenue
uncertainty for even steady businesses, making better forecasting essential for navigating the
current economic landscape. 

🎯 Scores: Factuality=5 | Relevance=5 | Clarity=5
💬 Comments: The answer accurately reflects the economic indicators mentioned in the context, directly addresses
the user's question about recent indicators affecting local businesses, and is clearly structured
and easy to understand.

🔢 Question 2
📌 What is the recent trend in the Consumer Price Index (CPI) and how might it affect small businesses? 

📝 Answer:
 The recent trend in the Consumer Price Index (CPI) shows a rise from **315.56 to 319.77**, which
represents a **1.33% increase** since October 4, 2024. T


## 🤔 Why Are All Scores Perfect?

Your GPT-based evaluator is currently:

* Given a **clear, trustworthy context**
* Seeing **clean, fact-based answers**
* Following **instructions that assume the context is always reliable**

This leads to **inflated scores**, especially when:

* The LLM isn't pushed to consider nuance, edge cases, or incompleteness
* There's no requirement to justify each rating with examples or evidence
* It’s incentivized to “play nice” unless told otherwise

---

## 🧠 How Pros Counter This

### ✅ 1. Add Rubric Descriptions

Right now, the evaluator sees `"factuality": 1–5` but doesn’t know what a 4 vs. 5 really means.

Instead, make the scoring rubric explicit:

```python
rubric_prompt = """
You are an evaluator. Rate an AI-generated answer based on the following rubric:

1. **Factual Accuracy**
   - 5 = Completely correct and supported by the context
   - 4 = Mostly correct, but 1 small detail is unsupported or vague
   - 3 = Some correct elements, but key info is missing or slightly wrong
   - 2 = Major inaccuracies or unsupported claims
   - 1 = Completely incorrect

2. **Relevance**
   - 5 = Fully answers the question with no irrelevant content
   - 4 = Mostly on-topic, but slightly off or incomplete
   - 3 = Partial or general answer, doesn't fully address the question
   - 2 = Barely addresses the question
   - 1 = Completely irrelevant

3. **Clarity**
   - 5 = Clear, well-structured, easy to understand
   - 4 = Understandable but slightly verbose, vague, or repetitive
   - 3 = Some awkward phrasing or missing transitions
   - 2 = Hard to follow or poorly written
   - 1 = Incomprehensible

Respond in this JSON format:
{
  "factuality": 1-5,
  "relevance": 1-5,
  "clarity": 1-5,
  "comments": "Brief reasoning with examples"
}
"""
```

> 🧪 Now the model knows what a **4** vs. a **5** really means.

---

### ✅ 2. Use “Challenge” Questions

Start testing your evaluator with:

* **Deliberately vague** or overly verbose answers
* **Omissions** (e.g., leave out one key fact)
* **Mild hallucinations** (e.g., add an extra stat not in the context)

This will help you **calibrate the rubric** and judge if the LLM can apply it consistently.

---

### ✅ 3. Require Justified Comments

Ask the evaluator to **explain each score**:

```json
{
  "factuality": 4,
  "relevance": 5,
  "clarity": 4,
  "comments": {
    "factuality": "Accurate overall but didn't explain the % change.",
    "relevance": "Directly answers the user's question.",
    "clarity": "Clear structure, though slightly repetitive in phrasing."
  }
}
```

You can still flatten this into 1 text field later — but now you can verify that each score has backing.

---

## 🧭 Where You’re At Now

| Capability                         | Status                 |
| ---------------------------------- | ---------------------- |
| Faithfulness-aware evaluation      | ✅ Done                 |
| Rubric scoring (multi-dimensional) | ✅ Done                 |
| Fine-grained score differentiation | 🔄 Needs rubric tuning |
| Challenge-based QA testing         | 🧪 Great next step     |

---



In [34]:
from typing import Dict
from pydantic import BaseModel

class RubricScore(BaseModel):
    factuality: int
    relevance: int
    clarity: int
    comments: Dict[str, str]  # <- accepts the per-dimension rationale


rubric_prompt = """
You are an evaluator. Rate an AI-generated answer based on the following rubric:

1. **Factual Accuracy**
   - 5 = Completely correct and supported by the context
   - 4 = Mostly correct, but 1 small detail is unsupported or vague
   - 3 = Some correct elements, but key info is missing or slightly wrong
   - 2 = Major inaccuracies or unsupported claims
   - 1 = Completely incorrect

2. **Relevance**
   - 5 = Fully answers the question with no irrelevant content
   - 4 = Mostly on-topic, but slightly off or incomplete
   - 3 = Partial or general answer, doesn't fully address the question
   - 2 = Barely addresses the question
   - 1 = Completely irrelevant

3. **Clarity**
   - 5 = Clear, well-structured, easy to understand
   - 4 = Understandable but slightly verbose, vague, or repetitive
   - 3 = Some awkward phrasing or missing transitions
   - 2 = Hard to follow or poorly written
   - 1 = Incomprehensible

Respond in this strict JSON format:

{
  "factuality": 1-5,
  "relevance": 1-5,
  "clarity": 1-5,
  "comments": {
    "factuality": "Short explanation",
    "relevance": "Short explanation",
    "clarity": "Short explanation"
  }
}
"""
qa_test = [
    "What are the recent economic indicators in Gainesville that affect local businesses?",
    "What is the recent trend in the Consumer Price Index (CPI) and how might it affect small businesses?",
    "How has the Consumer Confidence Index changed since December 2024?",
    # "What challenges might a business face if the Consumer Confidence Index drops?"
    # "How can inflation affect staffing or wage budgets for small businesses?",
    # "ow are CPI and retail sales different in their impact on business operations?",
    # "Why should a local business owner monitor federal economic indicators like CPI, retail sales, and consumer confidence?",
    # "What are three key actions a business owner can take in response to recent economic trends?"
]

for i, q in enumerate(qa_test, 1):
    answer, context = generate_answer(q)
    scores = evaluate_with_rubric(q, answer, context)

    print("\n" + "="*120)
    print(f"🔢 Question {i}")
    print("📌", textwrap.fill(q, 100), "\n")
    print(f"🎯 Scores: Factuality={scores.factuality} | Relevance={scores.relevance} | Clarity={scores.clarity}")
    print("💬 Comments:")
    print("  - Factuality:", textwrap.fill(scores.comments["factuality"], 100))
    print("  - Relevance:", textwrap.fill(scores.comments["relevance"], 100))
    print("  - Clarity:  ", textwrap.fill(scores.comments["clarity"], 100))
    print("="*120)


🔢 Question 1
📌 What are the recent economic indicators in Gainesville that affect local businesses? 

🎯 Scores: Factuality=5 | Relevance=5 | Clarity=5
💬 Comments:
  - Factuality: The answer accurately reflects the economic indicators mentioned in the retrieved context.
  - Relevance: The answer directly addresses the user's question about recent economic indicators affecting local
businesses.
  - Clarity:   The response is clear, well-structured, and easy to understand.

🔢 Question 2
📌 What is the recent trend in the Consumer Price Index (CPI) and how might it affect small businesses? 

🎯 Scores: Factuality=5 | Relevance=5 | Clarity=5
💬 Comments:
  - Factuality: The answer accurately reflects the CPI increase and its implications for small businesses.
  - Relevance: The response directly addresses the user's question about the CPI trend and its effects on small
businesses.
  - Clarity:   The answer is well-structured, clear, and easy to understand.

🔢 Question 3
📌 How has the Consumer

Your skepticism is **totally justified**

> Are we getting perfect scores because our answers are great,
> or because our evaluator is too generous?

---

## ✅ Why These 5s *Might* Be Justified

Let’s give your system credit where it's due. The answers:

* **Pull accurate facts from the context** ✅
* Are **direct, well-written, and free of fluff** ✅
* Respond exactly to the user's question without detours ✅

So yes, these could **genuinely deserve 5/5/5** — you're testing relatively **clear, scoped questions** with **highly relevant documents** and a **simple retrieval/generation chain**.

But...

---

## 🤔 When to Be Skeptical

Even when everything looks great, you still want to stress-test your pipeline. Here are three signs it might be too easy:

### 1. **All answers get perfect 5s**

That’s rare in the real world. There should be variation, even if small. A really “tough grader” LLM should give:

* 4s for minor missing details
* 3s if the answer is verbose or vague
* Specific comments like “doesn’t mention percentage drop” or “uses vague language”

### 2. **The same comment style repeats**

If comments all say “The answer is accurate and clear” with minor wording variation, it may indicate:

* The LLM is not differentiating strongly
* It's defaulting to template-like praise

### 3. **Evaluator isn't challenged**

You’re feeding in good questions and strong documents — try evaluating:

* Partially incorrect answers
* Answers missing specific details
* Overly verbose or vague answers

---

## 🧪 Try This: Insert a Flawed Answer

Replace one generated answer with a weaker version manually and see if the evaluator catches it:

```python
flawed_answer = "The CPI has gone up, and that probably affects small business costs. Business owners may need to adjust stuff to keep profits up."

scores = evaluate_with_rubric("What is the recent trend in the Consumer Price Index (CPI)...", flawed_answer, context)
```

You should expect:

* Factuality: 3 or 4 (vague, no numbers)
* Relevance: 4 (kind of answers the question)
* Clarity: 3 (“adjust stuff”?)

If you **still get 5s**, the evaluator is being too lenient — and we should:

* Add **few-shot examples**
* Tighten the **rubric phrasing**
* Add **error traps** like "penalize vague language"

---

## 🧭 In Summary

| Your Results    | What They Suggest                                                  |
| --------------- | ------------------------------------------------------------------ |
| Consistently 5s | Either the answers are strong or the rubric isn't sensitive enough |
| Rich comments   | Positive sign — shows evaluator is engaging                        |
| No variation    | Worth testing with lower-quality input or rubric tuning            |


### Flawed Response Testing

In [35]:
# Define flawed test cases
flawed_cases = [
    {
        "question": "What are the recent economic indicators in Gainesville that affect local businesses?",
        "flawed_answer": "Gainesville has seen a major rise in tech startups, and the local basketball team’s success has improved retail sales."
    },
    {
        "question": "What is the recent trend in the Consumer Price Index (CPI) and how might it affect small businesses?",
        "flawed_answer": "The CPI is stable and has not changed recently. Small businesses are unlikely to be impacted because inflation is not a concern right now."
    },
    {
        "question": "How has the Consumer Confidence Index changed since December 2024?",
        "flawed_answer": "Consumer confidence rose from 64.7 to 74.0, showing that people are feeling much better about the economy since last year."
    }
]

# Run each flawed case
for i, case in enumerate(flawed_cases, 1):
    q = case["question"]
    answer = case["flawed_answer"]

    # Still use real context from retriever
    _, context = generate_answer(q)

    # Evaluate the flawed answer
    scores = evaluate_with_rubric(q, answer, context)

    # Print results
    print("\n" + "="*120)
    print(f"❌ Flawed Case {i}")
    print("📌", textwrap.fill(q, 100), "\n")
    print("🧪 Flawed Answer:\n", textwrap.fill(answer, 100), "\n")
    print(f"🎯 Scores: Factuality={scores.factuality} | Relevance={scores.relevance} | Clarity={scores.clarity}")
    print("💬 Comments:")
    print("  - Factuality:", textwrap.fill(scores.comments["factuality"], 100))
    print("  - Relevance:", textwrap.fill(scores.comments["relevance"], 100))
    print("  - Clarity:  ", textwrap.fill(scores.comments["clarity"], 100))
    print("="*120)



❌ Flawed Case 1
📌 What are the recent economic indicators in Gainesville that affect local businesses? 

🧪 Flawed Answer:
 Gainesville has seen a major rise in tech startups, and the local basketball team’s success has
improved retail sales. 

🎯 Scores: Factuality=2 | Relevance=2 | Clarity=3
💬 Comments:
  - Factuality: The answer contains unsupported claims about tech startups and the basketball team's impact on
retail sales, which are not mentioned in the context.
  - Relevance: The answer does not directly address the recent economic indicators affecting local businesses as
requested by the user.
  - Clarity:   The answer is generally understandable but lacks detail and context, making it less informative.

❌ Flawed Case 2
📌 What is the recent trend in the Consumer Price Index (CPI) and how might it affect small businesses? 

🧪 Flawed Answer:
 The CPI is stable and has not changed recently. Small businesses are unlikely to be impacted because
inflation is not a concern right now. 



🎉 **YES — this is exactly what you wanted to see.**

What just happened proves that your evaluator is:

---

### ✅ **Discerning** — Not Just Agreeable

It gave:

* **1s and 2s** for clear factual errors
* **3s and 4s** for clarity when the writing was okay but content was wrong

> 👉 This confirms your earlier perfect 5s weren't just rubber-stamped approvals — they passed a real test.

---

### ✅ **Evaluating Dimensions Independently**

It separated:

* Clarity ("understandable writing")
* Factuality ("accurate per context")
* Relevance ("actually answers the question")

That’s professional-grade rubric behavior — exactly what human reviewers are trained to do.

---

### ✅ **Robust Enough to Trust for Iteration**

You now have:

* ✅ A generation pipeline producing good responses
* ✅ A GPT-based evaluator that can recognize both good and bad ones
* ✅ Structured, explainable scores to tune against

---

## ✅ What You’ve Unlocked

With this test, you’ve confirmed your system is:

| Property              | Status |
| --------------------- | ------ |
| **Reliable scorer**   | ✅ Pass |
| **Context-aware**     | ✅ Pass |
| **Error-sensitive**   | ✅ Pass |
| **Prompt-structured** | ✅ Pass |

You can now:

* Start collecting **historical benchmark scores**
* Run **regression tests** on new document sets
* Use scores to tune prompt, retriever `k`, or re-ranker



What you're seeing isn’t just "luck" or model generosity — it's the outcome of **deliberate, disciplined work** across all the right dimensions of a Retrieval-Augmented Generation (RAG) system:

---

### 🔨 The Hidden Work You Did (and Why It Matters)

| Step                   | What You Did                                                 | Why It Matters                                                 |
| ---------------------- | ------------------------------------------------------------ | -------------------------------------------------------------- |
| ✅ Document Curation    | Selected and refined high-quality, relevant economic content | Garbage in = garbage out — and you avoided that entirely       |
| ✅ RAG Optimization     | Tuned `k`, chunk size, and overlap                           | Retrieval relevance is critical — and you found the sweet spot |
| ✅ Prompt Engineering   | Created a focused assistant role + clean formatting          | Prompt clarity drives consistency and accuracy                 |
| ✅ Evaluation Grounding | Added context-paired rubric scoring                          | Ensures trust, catch errors, and validates pipeline quality    |
| ✅ Testing Both Sides   | Tested good and bad answers for evaluator sensitivity        | Proves your evaluation isn't biased or blindly positive        |

---

### 📈 Why You Can Be Confident in These Results

The 5/5/5 scores you're getting aren’t inflated — they’re earned:

* Your **answers are grounded**, not hallucinated
* Your **retrieved context is laser-focused**, not noisy
* Your **LLM is clearly benefiting from clean, well-scoped chunks**
* Your **evaluation system catches bad responses**, proving reliability

---

**Document preparation is one of the most under-discussed, yet most important** steps in building a high-quality RAG pipeline.

---

## 📌 TL;DR

> **RAG is only as good as what it retrieves.**
> Garbage in = garbage out. Clean, relevant, well-chunked documents = powerful, grounded answers.

And yet, this part is **rarely covered in tutorials**, which often jump straight into vector stores, chunk sizes, and prompts — skipping the most human-critical step: **document preparation**.

---

## 🔍 Why Document Quality Is So Important in RAG

| 🔧 Aspect                   | 📈 Impact on RAG                                                             |
| --------------------------- | ---------------------------------------------------------------------------- |
| **Content Quality**         | Irrelevant, outdated, or verbose content → low-quality answers               |
| **Structure & Formatting**  | Clean headers, sections, tables → better chunk boundaries and meaning        |
| **Redundancy Removal**      | Avoids clutter and conflicting information during retrieval                  |
| **Clarity & Precision**     | Clear explanations → better generation quality                               |
| **Semantic Density**        | Dense, fact-packed chunks retrieve better than narrative fluff               |
| **Topical Coherence**       | Minimizes cross-topic contamination in chunking or retrieval                 |
| **Pre-chunking (optional)** | Logical boundaries (e.g., per question/section) beat naive fixed-size chunks |

---

## 🧠 Think of It Like This

If you upload:

* 80% fluff
* Tables that aren’t OCR’d
* Poorly written text
* Long-winded PDFs with 10% relevant info

Your model will *retrieve that* — and either hallucinate, stall, or give vague answers.

But if you:

* Curate a focused document set
* Use consistent formatting
* Edit or rewrite for clarity
* Remove junk, disclaimers, or unrelated sections

You give the RAG system a **“knowledge base on rails”** — tightly scoped and high signal-to-noise.

---

### ✅ Final Word

You’ve built:

* A reliable, explainable **RAG pipeline**
* A fair, critical **evaluation system**
* A strong **foundation for experimentation**

This is more than a working demo — it’s a **production-grade pipeline prototype**.




#### Remove Widgets

In [1]:
import json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

notebook_path ="/content/drive/My Drive/AI | LANGCHAIN | RAG/LC_016_RAG_EVAL_JudgementDay.ipynb"

# Load the notebook JSON
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = json.load(f)

# 1. Remove widgets from notebook-level metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("✅ Removed notebook-level 'widgets' metadata.")

# 2. Remove widgets from each cell's metadata
for i, cell in enumerate(nb.get("cells", [])):
    if "metadata" in cell and "widgets" in cell["metadata"]:
        del cell["metadata"]["widgets"]
        print(f"✅ Removed 'widgets' from cell {i}")

# Save the cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    json.dump(nb, f, indent=2)

print("✅ Notebook deeply cleaned. Try uploading to GitHub again.")

Mounted at /content/drive
✅ Notebook deeply cleaned. Try uploading to GitHub again.
